In [36]:
# !pip install selenium

In [49]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait

import re
import pandas as pd

In [56]:
links = []

for i in range(1,12):
    driver = webdriver.Chrome('/opt/miniconda3/bin/chromedriver_mac_arm64/chromedriver')
    if i == 1:
        driver.get('https://nab.banklocationmaps.com.au/en/branches/aus/victoria/')
    else:
        driver.get(f'https://nab.banklocationmaps.com.au/en/branches/aus/victoria?page={i}')
    
    directions = driver.find_elements(By.XPATH, "//a[@class='btn-sm btn-outline-secondary btn-shadow btn-block']")
    for d in directions:
        links.append(d.get_attribute("href"))

    driver.close()

/var/folders/95/76mmy6pn2kvccmkyykdt6tvw0000gn/T/ipykernel_94319/3256854471.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('/opt/miniconda3/bin/chromedriver_mac_arm64/chromedriver')


In [65]:
all_branch = {}
def get_longlat(locs, text):
    branch = ''
    for line in text.split('\n'):
        if '"streetAddress"' in line:
            branch = line.split(': ')[1][1:-2]
            locs[branch] = {'long':0, 'lat': 0}
        if '"longitude"' in line:
            print(line.split(': ')[1])
            locs[branch]['long'] = float(line.split(': ')[1])
        elif '"latitude"' in line:
            locs[branch]['lat'] = float(line.split(': ')[1][:-1])
    return locs

In [ ]:
vic_branches = [l for l in links if l[-9:] == 'locations']
for link in vic_branches:
    driver = webdriver.Chrome('/opt/miniconda3/bin/chromedriver_mac_arm64/chromedriver')
    driver.get(link)
    geo = driver.page_source
    all_branch = get_longlat(all_branch, geo)
    driver.close()
if len(all_branch) != len(vic_branches):
    print(f"""Warning - Number of extracted NAB Victorian branches {len(all_branch)}, 
          expected {len(vic_branches)} branches!""")

In [94]:
# Check for duplicated branch names
len(set([re.search("nab-branch-(.*)?directions=1", i).group(0) for i in vic_branches]))

180

In [ ]:
# sorted([ i for i in all_branch.keys()])

In [82]:
# Get duplicates based on long lat locations
dups = []
for k1, v1 in all_branch.items():
    for k2, v2 in all_branch.items():
        if k2 != k1 and v2 == v1 and k1 not in dups:
            dups.append(k2)

In [83]:
sorted(dups)

['100 Exhibition Street, Melbourne, VIC, 3000',
 '107 Main St, Mornington, VIC, 3931',
 '2 Eaton Street, Oakleigh, VIC, 3166',
 '224-228 Smith Street, Collingwood, VIC, 3066',
 '28 Hartnett Drive, Seaford, VIC, 3198',
 '293 Spring St, Reservoir, VIC, 3073',
 '406 Sydney Rd, Coburg, VIC, 3058',
 '566 Burke Road, Camberwell, VIC, 3124',
 '82-88 Franklin St, Traralgon, VIC, 3844',
 'Melbourne NAB House, 500 Bourke Street, Melbourne NAB House, Melbourne, VIC, 3000',
 'The Concourse, 700 Bourke St, The Concourse, Docklands, VIC, 3008']

In [87]:
# Remove duplicate branches
final_branch_set = {k: all_branch[k] for k in all_branch.keys() if k not in dups}

In [89]:
len(final_branch_set)

169

In [101]:
pd.DataFrame.from_dict(final_branch_set, orient='index')\
  .reset_index(names='branch')\
  .to_csv('branch_locations.csv', index=False)

,branch,long,lat
0,"Westfield Fountain Gate, 352 Princes Highway, ...",145.304441,-38.018206
1,"330 Collins St, Melbourne, VIC, 3000",144.963217,-37.816309
2,"460 Collins St, Melbourne, VIC, 3000",144.958852,-37.817673
3,"288 Lygon Street, Carlton, VIC, 3053",144.967263,-37.800216
4,"22 Watton St, Werribee, VIC, 3030",144.661003,-37.901344
...,...,...,...
164,"40 Federal St, Rainbow, VIC, 3424",141.994995,-35.898205
165,"16 Gilbert Street, Torquay, VIC, 3228",144.324342,-38.332092
166,"299-301 Esplanade, Lakes Entrance, VIC, 3909",147.983521,-37.881281
167,"Midtown Plaza, 186 Swanston Street, Midtown Pl...",144.965472,-37.813131
